# Ejercicio Percentil 95 NNA vs UDS

In [5]:
import pandas as pd
import numpy as np
import folium

In [2]:
def calculo_distancia(df, lat1,lon1,lat2,lon2):
    #Cálculo de distancia harvesiana
    lat1=df[lat1].astype(float)
    long1=df[lon1].astype(float)

    lat2=df[lat2].astype(float)
    long2=df[lon2].astype(float)
    long1, lat1, long2, lat2 = map(np.radians, [long1, lat1, long2, lat2])
    # haversine formula
    dlong = long2 - long1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlong / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    earth_radius = 6367
    d = earth_radius * c
    return df.assign(Distancia = lambda x : d)

In [3]:

base_final = pd.read_csv("./Data/Ejercicio Distancias NNA vs UDS.csv", sep="|", low_memory=False)
uds =  pd.read_excel("./Data/uds_agosto2021estadogeo1.xlsx", sheet_name="uds_agosto2021estadogeo1")

base_final['Distancia_mts']= base_final['Distancia'] * 1000

## Caso Cabecera percentil 95

In [19]:
caso_95pctile = base_final.loc[(base_final.pctile==95) & (base_final.Coord_NNA_en_Mpio=="Si")].sort_values(by="Distancia")
caso_95pctile = caso_95pctile[['idbeneficiario','CodigoDepartamentoResidenciaBeneficiario','DepartamentoResidenciaBeneficiario','CodigoMunicipioResidenciaBeneficiario','MunicipioResidenciaBeneficiario','ZonaUbicacionBeneficiario','CodigoUnidadServicio','UnidadServicio','NombreServicio','LATITUD','LONGITUD','LATITUD_NNA_2','LONGITUD_NNA_2','Distancia_mts','pctile','Coord_NNA_en_Mpio']]
caso_95pctile = caso_95pctile.loc[caso_95pctile.MunicipioResidenciaBeneficiario=="Bogota, D.C."].head(1).reset_index()
uds['lat_2'] = caso_95pctile['LATITUD_NNA_2'][0]
uds['lon_2'] = caso_95pctile['LONGITUD_NNA_2'][0]
uds = calculo_distancia(uds,'LATITUD', 'LONGITUD','lat_2','lon_2')

uds_caso_95pctile = uds.loc[uds.Distancia < 10].sort_values(by="Distancia",ascending=True).copy()

El color rojo se usa para el Beneficiario, el verde para su UDS acual y el azul todas las UDS a menos de 10 km

In [20]:
m = folium.Map(location=[caso_95pctile["LATITUD_NNA_2"],caso_95pctile["LONGITUD_NNA_2"]], zoom_start=10, tiles='CartoDB positron') 
folium.Marker(location=[caso_95pctile["LATITUD_NNA_2"],caso_95pctile["LONGITUD_NNA_2"]], popup='Beneficiario ID: {}'.format(caso_95pctile['idbeneficiario'][0]), draggable=False, icon=folium.Icon(color='red')).add_to(m)
folium.Marker(location=[caso_95pctile["LATITUD"],caso_95pctile["LONGITUD"]], popup='Código Unidad Servicio Actual: {}, Unidad Servicio Actual: {}'.format(caso_95pctile['CodigoUnidadServicio'][0], caso_95pctile['UnidadServicio'][0]), draggable=False, icon=folium.Icon(color='green')).add_to(m)
points = [[caso_95pctile["LATITUD_NNA_2"][0],caso_95pctile["LONGITUD_NNA_2"][0]],[caso_95pctile["LATITUD"][0], caso_95pctile["LONGITUD"][0]]]
folium.PolyLine(points, color='red').add_to(m)

for _, r in uds_caso_95pctile.iterrows():
    folium.Marker(location=[r["LATITUD"], r["LONGITUD"]], popup='Código UDS: {} <br> UDS: {}'.format(r['CodigoUnid'], r['UnidadServ']), draggable=False).add_to(m)

m

Beneficiario

In [23]:
caso_95pctile

,index,idbeneficiario,CodigoDepartamentoResidenciaBeneficiario,DepartamentoResidenciaBeneficiario,CodigoMunicipioResidenciaBeneficiario,MunicipioResidenciaBeneficiario,ZonaUbicacionBeneficiario,CodigoUnidadServicio,UnidadServicio,NombreServicio,LATITUD,LONGITUD,LATITUD_NNA_2,LONGITUD_NNA_2,Distancia_mts,pctile,Coord_NNA_en_Mpio
0,908710,20928088,11.0,Bogota D.C.,11001,"Bogota, D.C.",CABECERA,110011120955,LOS NIÑOS DEL FUTURO,HCB - COMUNITARIO,4.501944,-74.110278,4.183838,-74.21469,37194.707967,95.0,Si


UDS a menos de 10 km

In [22]:
uds_caso_95pctile

,OBJECTID_12,CodigoDepa,Departamen,CodigoMuni,MunicipioU,CodigoUnid,UnidadServ,NombreServ,EstadoUDS,LatitudUDS,...,CODMUN,MUNICIPI_2,area_km,estadogeo,OID_,CodigoUn_1,Cnt_Codigo,lat_2,lon_2,Distancia
13213,13214,11,Bogota D.C.,11001,"Bogota, D.C.",1100100015305,MUNDO CURIOSO 2,HCB - COMUNITARIO,ACTIVA,4.10.4. N,...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,10977,1.100100e+12,1,4.183838,-74.21469,5.497112
14234,14235,11,Bogota D.C.,11001,"Bogota, D.C.",110011113962,LOS POLLUELOS,HCB AGRUPADOS - COMUNITARIO,ACTIVA,4.7.17. N,...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,13961,1.100111e+11,1,4.183838,-74.21469,7.027097
15111,15112,11,Bogota D.C.,11001,"Bogota, D.C.",110011144025,EL ARTE DE APRENDER,HCB AGRUPADOS - COMUNITARIO,ACTIVA,4.7.13. N,...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,15149,1.100111e+11,1,4.183838,-74.21469,7.756482
12982,12983,11,Bogota D.C.,11001,"Bogota, D.C.",1100100135276,MI BELLO FUTURO,HCB - COMUNITARIO,ACTIVA,"4.6.39,80. N",...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,12614,1.100100e+12,1,4.183838,-74.21469,8.102200
15255,15256,11,Bogota D.C.,11001,"Bogota, D.C.",110011113706,OSITO YOGUI.,HCB - COMUNITARIO,ACTIVA,"4.15.00,00. N",...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,13901,1.100111e+11,1,4.183838,-74.21469,8.132394
14893,14894,11,Bogota D.C.,11001,"Bogota, D.C.",1100100135268,MIS QUERIDOS CHIQUILLOS,HCB - COMUNITARIO,ACTIVA,"4.6.38,16. N",...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,12612,1.100100e+12,1,4.183838,-74.21469,8.497676
15064,15065,11,Bogota D.C.,11001,"Bogota, D.C.",110011140800,HOGAR INFANTIL GERONA 2017,HOGARES INFANTILES - INSTITUCIONAL,ACTIVA,4.6.27. N,...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,15128,1.100111e+11,1,4.183838,-74.21469,8.510389
11991,11992,11,Bogota D.C.,11001,"Bogota, D.C.",1100100124064,CDI SANTO DOMINGO SAVIO SEDE BACHUÉ,CDI SIN ARRIENDO - INSTITUCIONAL,ACTIVA,"4.7.1,48. N",...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,11729,1.100100e+12,1,4.183838,-74.21469,8.569101
11859,11860,11,Bogota D.C.,11001,"Bogota, D.C.",1100100051188,JUGANDO APRENDEMOS,HCB - COMUNITARIO,ACTIVA,4.7.06. N,...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,11448,1.100100e+12,1,4.183838,-74.21469,8.753011
20308,20309,11,Bogota D.C.,11001,"Bogota, D.C.",1100100055324,FORMACIÓN PARA LA VIDA,HCB - COMUNITARIO,ACTIVA,4.7.09. N,...,11001.0,"BOGOTÁ, D.C.",0,georeferenciada agosto 31,11473,1.100100e+12,1,4.183838,-74.21469,8.816498


## Caso Percentil 95 Zona Ubicación Resto

In [26]:
caso_95pctile = base_final.loc[(base_final.pctile==95) & (base_final.Coord_NNA_en_Mpio=="Si")& (base_final.ZonaUbicacionBeneficiario=="RESTO")].sort_values(by="Distancia")
caso_95pctile = caso_95pctile[['idbeneficiario','CodigoDepartamentoResidenciaBeneficiario','DepartamentoResidenciaBeneficiario','CodigoMunicipioResidenciaBeneficiario','MunicipioResidenciaBeneficiario','ZonaUbicacionBeneficiario','CodigoUnidadServicio','UnidadServicio','NombreServicio','LATITUD','LONGITUD','LATITUD_NNA_2','LONGITUD_NNA_2','Distancia_mts','pctile','Coord_NNA_en_Mpio']]
caso_95pctile = caso_95pctile.loc[caso_95pctile.MunicipioResidenciaBeneficiario=="Bajo Baudo"].head(1).reset_index()
uds['lat_2'] = caso_95pctile['LATITUD_NNA_2'][0]
uds['lon_2'] = caso_95pctile['LONGITUD_NNA_2'][0]
uds = calculo_distancia(uds,'LATITUD', 'LONGITUD','lat_2','lon_2')

uds_caso_95pctile = uds.loc[uds.Distancia < 15].sort_values(by="Distancia",ascending=True).copy()

In [18]:
m = folium.Map(location=[caso_95pctile["LATITUD_NNA_2"],caso_95pctile["LONGITUD_NNA_2"]], zoom_start=10, tiles='CartoDB positron') 
folium.Marker(location=[caso_95pctile["LATITUD_NNA_2"],caso_95pctile["LONGITUD_NNA_2"]], popup='Beneficiario ID: {}'.format(caso_95pctile['idbeneficiario'][0]), draggable=False, icon=folium.Icon(color='red')).add_to(m)
folium.Marker(location=[caso_95pctile["LATITUD"],caso_95pctile["LONGITUD"]], popup='Código Unidad Servicio Actual: {}, Unidad Servicio Actual: {}'.format(caso_95pctile['CodigoUnidadServicio'][0], caso_95pctile['UnidadServicio'][0]), draggable=False, icon=folium.Icon(color='green')).add_to(m)
points = [[caso_95pctile["LATITUD_NNA_2"][0],caso_95pctile["LONGITUD_NNA_2"][0]],[caso_95pctile["LATITUD"][0], caso_95pctile["LONGITUD"][0]]]
folium.PolyLine(points, color='red').add_to(m)

for _, r in uds_caso_95pctile.iterrows():
    folium.Marker(location=[r["LATITUD"], r["LONGITUD"]], popup='Código UDS: {} <br> UDS: {}'.format(r['CodigoUnid'], r['UnidadServ']), draggable=False).add_to(m)

m

Beneficiario

In [27]:
caso_95pctile

,index,idbeneficiario,CodigoDepartamentoResidenciaBeneficiario,DepartamentoResidenciaBeneficiario,CodigoMunicipioResidenciaBeneficiario,MunicipioResidenciaBeneficiario,ZonaUbicacionBeneficiario,CodigoUnidadServicio,UnidadServicio,NombreServicio,LATITUD,LONGITUD,LATITUD_NNA_2,LONGITUD_NNA_2,Distancia_mts,pctile,Coord_NNA_en_Mpio
0,384048,16650939,27.0,Chocó,27077,Bajo Baudo,RESTO,270771135752,PUERTO PIÑA,ATENCIÓN PROPIA E INTERCULTURAL,4.921667,-77.363611,4.832472,-77.034802,37731.748102,95.0,Si


UDS a menos de 15 KM

In [28]:
uds_caso_95pctile

,OBJECTID_12,CodigoDepa,Departamen,CodigoMuni,MunicipioU,CodigoUnid,UnidadServ,NombreServ,EstadoUDS,LatitudUDS,...,CODMUN,MUNICIPI_2,area_km,estadogeo,OID_,CodigoUn_1,Cnt_Codigo,lat_2,lon_2,Distancia
32430,32431,27,Chocó,27077,Bajo Baudo,270771127514,RUBELIA CABEZON SOBRICAMA,HCB - COMUNITARIO,ACTIVA,4.47.28. N,...,27077.0,BAJO BAUDÓ,0,georeferenciada agosto 31,38046,2.707711e+11,1,4.832472,-77.034802,6.727455
38300,38301,27,Chocó,27077,Bajo Baudo,270771147535,MIRALIA CONQUISTA ISMARE,HCB - COMUNITARIO,ACTIVA,4.47.28. N,...,27077.0,BAJO BAUDÓ,0,georeferenciada agosto 31,38092,2.707711e+11,1,4.832472,-77.034802,6.727455
40854,40855,27,Chocó,27077,Bajo Baudo,2707700094565,KATTY MEMBACHE SOBRICAMA,HCB - COMUNITARIO,ACTIVA,4.47.28. N,...,27077.0,BAJO BAUDÓ,0,georeferenciada agosto 31,38026,2.707700e+12,1,4.832472,-77.034802,6.727455
63552,63553,27,Chocó,27077,Bajo Baudo,270771120581,ELINDA MEMBACHE CARPIO,HCB - COMUNITARIO,ACTIVA,"4.47.28,49. N",...,27077.0,BAJO BAUDÓ,0,georeferenciada agosto 31,38042,2.707711e+11,1,4.832472,-77.034802,6.732893
41266,41267,27,Chocó,27077,Bajo Baudo,2707700113153,SOMALIA CHAMARRA ORTIZ,HCB - COMUNITARIO,ACTIVA,"4.47.28,493. N",...,27077.0,BAJO BAUDÓ,0,georeferenciada agosto 31,38040,2.707700e+12,1,4.832472,-77.034802,6.733281
36457,36458,27,Chocó,27077,Bajo Baudo,270771135803,UNIÓN PITALITO,ATENCIÓN PROPIA E INTERCULTURAL,ACTIVA,4.44.30. N,...,27077.0,BAJO BAUDÓ,0,georeferenciada agosto 31,38065,2.707711e+11,1,4.832472,-77.034802,11.718693
64459,64460,27,Chocó,27430,Medio Baudo,274301151852,ALBACA,ATENCIÓN PROPIA E INTERCULTURAL,ACTIVA,4.42.16. N,...,27361.0,ISTMINA,0,GEOREFERENCIADO MAYO 15,38636,2.743012e+11,1,4.832472,-77.034802,14.317669
